# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 22 2022 2:02PM,240222,18,CLRSPC-4515205235,"ClearSpec, LLC",Released
1,Jun 22 2022 1:59PM,240221,15,8179041,FUJIFILM Diosynth Biotechnologies Texas LLC,Released
2,Jun 22 2022 1:58PM,240220,18,CLRSPC-4515205281,"ClearSpec, LLC",Released
3,Jun 22 2022 1:54PM,240219,19,ADV80003504,"AdvaGen Pharma, Ltd",Released
4,Jun 22 2022 1:54PM,240219,19,ADV80003505,"AdvaGen Pharma, Ltd",Released
5,Jun 22 2022 1:54PM,240219,19,ADV80003506,"AdvaGen Pharma, Ltd",Released
6,Jun 22 2022 1:54PM,240219,19,ADV80003507,"AdvaGen Pharma, Ltd",Released
7,Jun 22 2022 1:54PM,240219,19,ADV80003508,"AdvaGen Pharma, Ltd",Released
8,Jun 22 2022 1:54PM,240219,19,ADV80003509,"AdvaGen Pharma, Ltd",Released
9,Jun 22 2022 1:54PM,240219,19,ADV80003510,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
48,240218,Released,1
49,240219,Released,21
50,240220,Released,1
51,240221,Released,1
52,240222,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240218,NaN,NaN,1.0
240219,NaN,NaN,21.0
240220,NaN,NaN,1.0
240221,NaN,NaN,1.0
240222,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240124,0.0,1.0,0.0
240133,7.0,17.0,14.0
240134,0.0,0.0,1.0
240135,0.0,0.0,1.0
240137,7.0,10.0,15.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240124,0,1,0
240133,7,17,14
240134,0,0,1
240135,0,0,1
240137,7,10,15


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240124,0,1,0
1,240133,7,17,14
2,240134,0,0,1
3,240135,0,0,1
4,240137,7,10,15


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240124,,1,
1,240133,7,17,14
2,240134,,,1
3,240135,,,1
4,240137,7,10,15


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 22 2022 2:02PM,240222,18,"ClearSpec, LLC"
1,Jun 22 2022 1:59PM,240221,15,FUJIFILM Diosynth Biotechnologies Texas LLC
2,Jun 22 2022 1:58PM,240220,18,"ClearSpec, LLC"
3,Jun 22 2022 1:54PM,240219,19,"AdvaGen Pharma, Ltd"
24,Jun 22 2022 1:51PM,240218,22,"NBTY Global, Inc."
25,Jun 22 2022 1:47PM,240194,16,Seqirus
26,Jun 22 2022 1:37PM,240214,10,Emerginnova
27,Jun 22 2022 1:22PM,240213,10,ISDIN Corporation
60,Jun 22 2022 1:20PM,240207,12,Hush Hush
63,Jun 22 2022 1:20PM,240205,15,FUJIFILM Diosynth Biotechnologies Texas LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 22 2022 2:02PM,240222,18,"ClearSpec, LLC",,,1
1,Jun 22 2022 1:59PM,240221,15,FUJIFILM Diosynth Biotechnologies Texas LLC,,,1
2,Jun 22 2022 1:58PM,240220,18,"ClearSpec, LLC",,,1
3,Jun 22 2022 1:54PM,240219,19,"AdvaGen Pharma, Ltd",,,21
4,Jun 22 2022 1:51PM,240218,22,"NBTY Global, Inc.",,,1
5,Jun 22 2022 1:47PM,240194,16,Seqirus,,,1
6,Jun 22 2022 1:37PM,240214,10,Emerginnova,,,1
7,Jun 22 2022 1:22PM,240213,10,ISDIN Corporation,,,33
8,Jun 22 2022 1:20PM,240207,12,Hush Hush,,,3
9,Jun 22 2022 1:20PM,240205,15,FUJIFILM Diosynth Biotechnologies Texas LLC,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 2:02PM,240222,18,"ClearSpec, LLC",1,,
1,Jun 22 2022 1:59PM,240221,15,FUJIFILM Diosynth Biotechnologies Texas LLC,1,,
2,Jun 22 2022 1:58PM,240220,18,"ClearSpec, LLC",1,,
3,Jun 22 2022 1:54PM,240219,19,"AdvaGen Pharma, Ltd",21,,
4,Jun 22 2022 1:51PM,240218,22,"NBTY Global, Inc.",1,,
5,Jun 22 2022 1:47PM,240194,16,Seqirus,1,,
6,Jun 22 2022 1:37PM,240214,10,Emerginnova,1,,
7,Jun 22 2022 1:22PM,240213,10,ISDIN Corporation,33,,
8,Jun 22 2022 1:20PM,240207,12,Hush Hush,3,,
9,Jun 22 2022 1:20PM,240205,15,FUJIFILM Diosynth Biotechnologies Texas LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 2:02PM,240222,18,"ClearSpec, LLC",1,,
1,Jun 22 2022 1:59PM,240221,15,FUJIFILM Diosynth Biotechnologies Texas LLC,1,,
2,Jun 22 2022 1:58PM,240220,18,"ClearSpec, LLC",1,,
3,Jun 22 2022 1:54PM,240219,19,"AdvaGen Pharma, Ltd",21,,
4,Jun 22 2022 1:51PM,240218,22,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 2:02PM,240222,18,"ClearSpec, LLC",1.0,NaN,NaN
1,Jun 22 2022 1:59PM,240221,15,FUJIFILM Diosynth Biotechnologies Texas LLC,1.0,NaN,NaN
2,Jun 22 2022 1:58PM,240220,18,"ClearSpec, LLC",1.0,NaN,NaN
3,Jun 22 2022 1:54PM,240219,19,"AdvaGen Pharma, Ltd",21.0,NaN,NaN
4,Jun 22 2022 1:51PM,240218,22,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 2:02PM,240222,18,"ClearSpec, LLC",1.0,0.0,0.0
1,Jun 22 2022 1:59PM,240221,15,FUJIFILM Diosynth Biotechnologies Texas LLC,1.0,0.0,0.0
2,Jun 22 2022 1:58PM,240220,18,"ClearSpec, LLC",1.0,0.0,0.0
3,Jun 22 2022 1:54PM,240219,19,"AdvaGen Pharma, Ltd",21.0,0.0,0.0
4,Jun 22 2022 1:51PM,240218,22,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3842919,72.0,9.0,10.0
12,240207,3.0,0.0,0.0
15,1681195,37.0,45.0,20.0
16,720589,3.0,1.0,0.0
18,480442,2.0,0.0,0.0
19,1441054,25.0,1.0,0.0
20,720560,11.0,8.0,0.0
21,480408,2.0,0.0,0.0
22,240218,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3842919,72.0,9.0,10.0
1,12,240207,3.0,0.0,0.0
2,15,1681195,37.0,45.0,20.0
3,16,720589,3.0,1.0,0.0
4,18,480442,2.0,0.0,0.0
5,19,1441054,25.0,1.0,0.0
6,20,720560,11.0,8.0,0.0
7,21,480408,2.0,0.0,0.0
8,22,240218,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,72.0,9.0,10.0
1,12,3.0,0.0,0.0
2,15,37.0,45.0,20.0
3,16,3.0,1.0,0.0
4,18,2.0,0.0,0.0
5,19,25.0,1.0,0.0
6,20,11.0,8.0,0.0
7,21,2.0,0.0,0.0
8,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,72.0
1,12,Released,3.0
2,15,Released,37.0
3,16,Released,3.0
4,18,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,10.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,9.0,0.0,45.0,1.0,0.0,1.0,8.0,0.0,0.0
Released,72.0,3.0,37.0,3.0,2.0,25.0,11.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Completed,10.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,0.0,45.0,1.0,0.0,1.0,8.0,0.0,0.0
2,Released,72.0,3.0,37.0,3.0,2.0,25.0,11.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Completed,10.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,0.0,45.0,1.0,0.0,1.0,8.0,0.0,0.0
2,Released,72.0,3.0,37.0,3.0,2.0,25.0,11.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()